# LCEL 체인 만들기
- LCEL 기본 개념 확인
- 파이프(|) 연산자를 사용해서 LangChain의 여러 컴포넌트(모델, 파서, 프롬프트)를 연결하는 체인 구성
- Pydantic을 활용하여 LLM의 출력을 구조화


## 라이브러리 불러오기

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv

c:\Users\user\miniconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## API KEY 불러오기

In [2]:
load_dotenv()
model = ChatOpenAI(model="gpt-4o-mini")

## 모델 호출

In [3]:
messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

model.invoke(messages)

AIMessage(content='안녕하세요, 개스톤! 당신의 제안이 정말 매력적이네요. 하지만 저는 오늘 저녁을 함께 하는 것보다는 다른 사람들과의 소중한 시간을 보내고 싶어요. 당신의 마음은 고마워요. 다른 날에 함께 할 기회가 있을지도 모르니, 너무 실망하지 마세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 62, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY10M48Nx2DKRLk394ZjFdl7vbnhC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--bb388fe5-6d64-45ef-ba52-c9b2cc6b1b8f-0', usage_metadata={'input_tokens': 62, 'output_tokens': 72, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser() #모델의 출력에서 문자열 내용만 추출하는 파서

result = model.invoke(messages)
parser.invoke(result) # AIMessage 객체가 아닌 순수 문자열 출력

'안녕하세요, 개스톤. 당신의 제안은 정말 매력적이지만, 저는 다른 방식의 사랑을 찾고 있어요. 저녁 대신 친구와 함께 나가서 시간을 보내는 건 어떨까요? 당신의 마음은 멋지지만, 저의 마음은 다른 곳에 있어요.'

## 체인 생성

In [5]:
# invoke의 입력이 model로 전달되고, model의 출력이 자동으로 parser로 전달
chain = model | parser
chain.invoke(messages)

'안녕하세요, 개스톤. 제안해 주셔서 감사하지만, 저는 당신과의 저녁 식사가 그리 맘에 들지 않아요. 저는 더 이상 고립된 삶을 원하지 않거든요. 제 마음에는 다른 사람들이 있어요. 우리와 다른 세계를 이해해 주길 바라요.'

## 프롬프트 템플릿
- {변수}를 넣어 쉽게 프롬프트를 만들 수 있음
- 딕셔너리 형태로 변수 값을 전달하면 완성된 프롬프트 생성

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


## LCEL 체인 심화

입력 (딕셔너리) -> prompt_template

프롬프트 (메시지) -> model

모델 출력 (AIMessage) -> parser

최종 결과 (문자열)

In [7]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'안녕하세요, 야수님! 저녁 함께하는 건 정말 좋은 생각이에요. 함께 보내는 시간이 어떤 모험이 될지 기대돼요. 어떤 음식을 좋아하시나요?'

## 체인 재사용

In [8]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

'안녕, 개스톤. 당신의 제안은 정말 매력적이지만, 저는 당신과 함께 하기보다는 제 마음에 드는 다른 일을 하고 싶어요. 저는 책을 읽는 것을 좋아하고, 자연 속에서 시간을 보내는 것도 좋아한답니다. 혹시 제게 다른 이야기를 나누고 싶다면 언제든지 해주세요.'

## 출력 구조화

### Pydantic 이란?

![이미지](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdna%2FbGANx3%2FbtrUuisjkwf%2FAAAAAAAAAAAAAAAAAAAAANsBUMSIh_Igw32SWHgX13XFytYN8IlVRpkRvRNn0_Vs%2Fimg.webp%3Fcredential%3DyqXZFxpELC7KVnFOS48ylbz2pIh7yKj8%26expires%3D1764514799%26allow_ip%3D%26allow_referer%3D%26signature%3DCu3WpMwN4tLwSv2cVQRNUdrTkcI%253D)

- 원하는 데이터의 스키마(구조)를 파이썬 클래스로 정의하게 해주는 라이브러리
- 런타임 타입 힌트 강제화
- 데이터 직렬화 역직렬화에 사용
- `BaseModel` : Pydantic 라이브러리에서 제공하는 기본 모델 클래스
  - 이 클래스를 상속하여 데이터 모델을 정의하고, 해당 모델의 필드와 유효성 검사 규칙을 설정할 수 있음
- ` Field` : 모델 필드를 정의할 때 사용
  - 필드를 통해 데이터 유효성 검사, 기본값 설정, 제약 조건 등을 세밀하게 제어

In [9]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib) # 모델에게 Adlib Pydantic 클래스의 구조를 반드시 따르도록 설정
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})

Adlib(answer='안녕하세요, 개스톤! 그렇게 말씀해 주셔서 감사합니다. 하지만 저는 당신이 생각하는 것보다 더 많은 것들을 원해요. 좋은 하루가 되었으면 해요!', main_emotion='기쁨', main_emotion_intensity=0.5)

### 문제 1)
BaseModel과 model.with_structured_output()을 함께 사용하는 이유를 적어보세요
A.LLM이 생성한 텍스트를 일관된 구조의 데이터로 안전하게 받을 수 있기 때문에